### הורדת מסמכי נוהל בנקאי תקין מבנק ישראל
כל המסמכין הורדו מהכתובת:
https://www.boi.org.il/roles/supervisionregulation/nbt/

In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import os
import json
from tqdm import tqdm
import time

In [2]:
def get_pdf_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            pdf_data = {}
            for row in soup.find_all('tr'):
                title_elem = row.find('td')
                date_elem = row.find('td', {'data-order': True})
                link_elem = row.find('a')

                # Check if all elements are found
                if title_elem and link_elem:
                    title = title_elem.get_text(strip=True)
                    file_name = link_elem.get_text(strip=True)
                    pdf_url = urljoin(url, link_elem['href'])
                    if date_elem:
                        date = date_elem.get_text(strip=True)
                    else:
                        date = None

                    # Use the file name as the key for the dictionary
                    pdf_data[file_name+".pdf"] = {
                        'title': title,
                        'date': date,
                        'pdf_url': pdf_url
                    }
            return pdf_data
        else:
            print(f"Failed to fetch HTML from {url}. Status code: {response.status_code}")
            return []
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []

In [10]:
def save_pdf(file_name, pdf_url, folder_path):
    try:
        response = requests.get(pdf_url)
        if response.status_code == 200:
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"PDF saved successfully: {file_path}")
            return True
        else:
            print(f"Failed to fetch PDF from {pdf_url}. Status code: {response.status_code}")
            return False
    except Exception as e:
        print(f"An error occurred while saving PDF: {str(e)}")
        return False

In [11]:
base_url = "https://www.boi.org.il/roles/supervisionregulation/nbt/"
folder_path = "pdf_folder"

In [12]:
# Create folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")

Folder 'PDF_folder2' created.


In [ ]:
start_time = time.time()
successful_saves = 0
pdf_data = get_pdf_data(base_url)
for file_name, pdf_info in tqdm(pdf_data.items(), desc="Saving PDFs"):
    pdf_url = pdf_info['pdf_url']
    print(file_name+" PDF URL:", pdf_url)
    if save_pdf(file_name, pdf_url, folder_path):
        successful_saves += 1
print(f"Total files saved successfully: {successful_saves} out of: {len(pdf_data.items())} urls  found in the website")
end_time = time.time()
print(f"Total time taken: {end_time - start_time} seconds")

In [83]:
with open('pdf_data2.json', 'w', encoding='utf-8') as json_file:
    json.dump(pdf_data, json_file, ensure_ascii=False)

In [84]:
with open('pdf_data.json', 'r', encoding='utf-8') as json_file:
    json_content = json_file.read()
    data = json.loads(json_content)
data


{'202.pdf': {'title': 'ההון הפיקוחי',
  'date': '04/02/2021',
  'pdf_url': 'https://www.boi.org.il/media/e3hc5jxy/202.pdf'},
 '203.pdf': {'title': 'הגישה הסטנדרטית - סיכון אשראי',
  'date': '15/08/2022',
  'pdf_url': 'https://www.boi.org.il/media/pmtpdcby/203_15.pdf'},
 '204.pdf': {'title': 'גישת הדירוגים הפנימיים לסיכון אשראי',
  'date': '21/10/2015',
  'pdf_url': 'https://www.boi.org.il/media/dekn2add/204.pdf'},
 '205.pdf': {'title': 'איגוח',
  'date': '30/05/2013',
  'pdf_url': 'https://www.boi.org.il/media/443hgn34/205.pdf'},
 '206.pdf': {'title': 'סיכון תפעולי',
  'date': '09/07/2012',
  'pdf_url': 'https://www.boi.org.il/media/wy0fpk15/206.pdf'},
 '208.pdf': {'title': 'סיכון שוק',
  'date': '20/06/2019',
  'pdf_url': 'https://www.boi.org.il/media/zsupfad2/208.pdf'},
 '211.pdf': {'title': 'הערכת נאותות הלימות ההון',
  'date': '30/05/2013',
  'pdf_url': 'https://www.boi.org.il/media/wlpjjxyf/211.pdf'},
 '301.pdf': {'title': 'דירקטוריון',
  'date': '23/02/2023',
  'pdf_url': 'https:

## vectorizing

In [73]:
from langchain_openai.embeddings.base import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
import os
import sys
import pinecone
import glob
import PyPDF2 
import json
from tqdm import tqdm

In [74]:
embeddings_model_name="text-embedding-3-small" # US$0.02 / 1M tokens
# embeddings_model_name="text-embedding-ada-002" # US$0.10 / 1M tokens
pdf_folder = "pdf_folder"

In [76]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
embeddings_model = OpenAIEmbeddings(model=embeddings_model_name, openai_api_key=openai_api_key)

In [ ]:
pc = Pinecone(api_key=pinecone_api_key)
pc.list_indexes()

In [78]:
index_name = 'test-index-1536'
if index_name not in pc.list_indexes().names():
    pc.create_index(
            index_name, 
            dimension=1536, 
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            ))
index = pc.Index(index_name)

In [80]:
def get_pdf_text(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
            text = text[0:230000] #limiting the text size
    return text

In [81]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        # separator=[" ", ",", "\n"],
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [82]:
def get_vector_embedding(text_chanks):
    openai_embedding = embeddings_model.embed_documents(text_chanks)
    return openai_embedding

In [38]:
with open('pdf_data.json', 'r', encoding='utf-8') as json_file:
    json_content = json_file.read()
    pdf_data = json.loads(json_content)

    for pdf_name in tqdm(pdf_data):
        pdf_path = os.path.join(pdf_folder, pdf_name)
        pdf_text = get_pdf_text(pdf_path)
        pdf_text_chunked = get_text_chunks(pdf_text)
        pdf_text_chunked_embedded = get_vector_embedding(pdf_text_chunked)
        chunk_ind_list =  [pdf_name+str(i) for i in range(len(pdf_text_chunked_embedded))]
        metadata = [{'text': paragraph} for paragraph in pdf_text_chunked]
    try:
        index.upsert(vectors=zip(chunk_ind_list,pdf_text_chunked_embedded,metadata))
    except Exception as e:
        error_code = getattr(e, 'code', None)  # Get the error code if available
        print(f"An error occurred during zipping. Error code: {error_code}. Error message: {e}")
        pass

  0%|          | 0/136 [00:00<?, ?it/s]

100%|██████████| 136/136 [01:18<00:00,  1.73it/s]


In [144]:
print('Total vector count:' ,index.describe_index_stats()['total_vector_count'])

Total vector count: 2620


### Deleteing index

In [70]:
# pc.delete_index(index_name)

# Retrieval Chain with LLM 

In [85]:
import os
from dotenv import load_dotenv
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser
import textwrap


In [163]:
embeddings_model_name="text-embedding-3-small" # US$0.02 / 1M tokens
# embeddings_model_name="text-embedding-ada-002" # US$0.10 / 1M tokens

model_mane ="gpt-4o"
# model_mane ="gpt-4-turbo"
# model_mane ="gpt-3.5-turbo"

In [164]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
embeddings_model = OpenAIEmbeddings(model=embeddings_model_name, openai_api_key=openai_api_key)
llm = ChatOpenAI(api_key=openai_api_key, model=model_mane, max_tokens=2048, temperature=0.1)

In [146]:
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [147]:
def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['Document'])

In [165]:
def main():
    # query = "איזה שאלות אפשר לשאול אותך?"
    # query = "האם חברת אשראי יכולה לתת אשראי במטבע חוץ"
    query = "מה הדרכים של חברות אשראי לגבות חוב?"
    vectorstore = Pinecone.from_existing_index(embedding=embeddings_model, index_name='test-index-1536')
    retriever =  vectorstore.as_retriever()
    qa_chain = RetrievalQA.from_chain_type(llm,retriever=retriever, return_source_documents=True, verbose=False)
    
    answer = qa_chain.invoke(query)
    # print(StrOutputParser(answer['result']))
    # print(answer)
    # print(answer['source_documents'][1])
    process_llm_response(answer)


In [166]:
main()

הדרכים של חברות אשראי לגבות חוב כוללות מספר אמצעים, כפי שמפורט בהוראות המפקח על הבנקים:

1. **ניסיונות גבייה ישירים**: חברות האשראי מצופות לנסות למצות דרכים שונות לגביית החוב ישירות מהלקוח לפני פנייה
לערכאות משפטיות. זה כולל פנייה ללקוח בדרכים שונות, כמו מכתבים, שיחות טלפון, והודעות דוא"ל.

2. **הליכים משפטיים**: אם הניסיונות לגבייה ישירה לא מצליחים, חברות האשראי רשאיות לנקוט בהליכים משפטיים נגד
הלקוח כדי לגבות את החוב.

3. **שימוש בביטחונות**: במקרים בהם יש ביטחונות שניתנו כנגד ההלוואה, חברות האשראי יכולות לממש את הביטחונות כדי
לגבות את החוב.

4. **שיתוף פעולה עם חברות גבייה**: חברות האשראי יכולות לשתף פעולה עם חברות גבייה מקצועיות המתמחות בגביית
חובות.

5. **הסדרי חוב**: חברות האשראי יכולות להציע ללקוח הסדרי חוב, כמו פריסת תשלומים או הנחות על חלק מהחוב, כדי להקל
על הלקוח להחזיר את החוב.

6. **מכירת החוב**: במקרים מסוימים, חברות האשראי יכולות למכור את החוב לחברות אחרות שמתמחות בגביית חובות.

הוראות אלו נועדו להבטיח שהליכי גביית החובות יתבצעו בצורה הוגנת ושקופה, תוך שמירה על זכויות הלק

KeyError: 'Document'

In [162]:
query = "מה הדרכים של חברות אשראי לגבות חוב?"
vectorstore = Pinecone.from_existing_index(embedding=embeddings_model, index_name='test-index-1536')
retriever =  vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm,retriever=retriever, return_source_documents=True, verbose=False)
answer = qa_chain.invoke(query)
print(answer)

{'query': 'מה הדרכים של חברות אשראי לגבות חוב?', 'result': 'הדרכים שבהן חברות אשראי יכולות לגבות חוב כוללות שימוש באמצעים שונים כדי להבטיח את החזר החוב מהלקוח. כאשר לקוח נקלע לקשיים כלכליים ואינו מסוגל לעמוד בתשלומים, החברה יכולה לנקוט בצעדים כדי לגבות את החוב, כולל:\n\n1. פנייה ישירה ללקוח: חברת האשראי יכולה לפנות ישירות ללקוח דרך מכתבים, שיחות טלפון או ביקורים כדי לדון במצבו הכלכלי ולמצוא פתרון לתשלום החוב.\n\n2. תוכניות תשלום: החברה יכולה להציע ללקוח תוכנית תשלום מותאמת אישית, שמאפשרת לו לשלם את החוב בקצב שהוא יכול לעמוד בו.\n\n3. הפנייה לגורמי גבייה: אם הלקוח לא משלם, החברה יכולה להעביר את החוב לגורמי גבייה מקצועיים שינסו לגבות את החוב באמצעות דרכים שונות.\n\n4. הליכים משפטיים: במקרה שכל הניסיונות האחרים נכשלים, חברת האשראי יכולה להגיש תביעה משפטית נגד הלקוח כדי לקבל פסק דין שיאפשר לה לגבות את החוב.\n\n5. פשרה או הסדר חוב: במקרים מסוימים, חברת האשראי עשויה להציע הסדר חוב או פשרה, שבה הלקוח משלם סכום נמוך יותר מהחוב המקורי כדי לסגור את החוב.\n\nחברות אשראי מחויבות לפעול בהוגנות ושקי

In [167]:
answer['source_documents']

[Document(page_content='Corporation (OPIC)  שהן סוכנויות של ממשלת ארה"ב, הערבה לכל תשלומי הקרן\nוהריבית של התחייבויותיהם, ובלבד  שניתן להיפרע מ מנה לאחר תשלום אשראי \nתעודות. \nשליטה משותפת \n6.  בטל. \nשותפות \n7.   חבות לתאגיד הבנקאי של שותפות (בין רשומה ובין בלתי רשומה), בה שותף לווה, תצורף\nלחבות הלווה אולם כאשר אחריותו של שותף בחבות מוגבלת, לא תעלה התוספת לחבות \nשל הלווה בגין השותפות על הסכום לו הוא אחראי". \nאשראי ללא זכות חזרה ללווה \n7א. על אף האמור בהוראה זו: \n(1) אשראי ללא זכות חזרה ללווה  בביטחון  ניירות ערך, יצורף הן לחבות הלווה והן \nלחבות התאגיד המנפיק את ניירות הערך. \n(2) בקבוצת לווים בה נכללים הן התאגיד הנרכש והן הלווים להם ניתנה החבות, תיכלל \nהחבות פעם אחת בלבד. \nלעניין סעיף זה, "אשראי  ללא זכות חזרה ללווה" - עסקת אשראי בה לתאגיד הבנקאי אין \nזכות להיפרע מכלל נכסי הלווה במקרה של כשל אשראי, אלא בעיקר מרכוש המשמש \nכביטחון בחוזה או רכוש ספציפי אחר, או כאשר הרכוש המשמש כביטחון הוא הנכס \nהעיקרי של הלווה. \nמיזוגים ורכישות \n8.   חל שינוי בחבות של לווה או של קבוצת לוו

### OLD shit

In [41]:
# query = "מהו נוהל בנקאי תקין?"
# query = "מבנקאות פתוחה בישראל?"
query = "האם חברת אשראי יכולה לסחור במטבע חוץ"
xq = embeddings_model.embed_query(query)
res = index.query(vector=[xq], top_k=8, include_metadata=True)
res

{'matches': [{'id': '359A.pdf3',
              'metadata': {'text': 'הם מעבירים פעילויות שונות למיקור חוץ, '
                                   'במטרה לצמצם את חשיפתם לסיכונים \n'
                                   'הפוטנציאליים הנובעים מכך, ובכלל זה: גיבוש '
                                   'מדיניות מיקור חוץ ות וכנית לניהול  \n'
                                   'הסיכונים במיקור חוץ, ביצוע בדיקת נאותות '
                                   'לנותן השירות, חתימה על חוזה מיקור  חוץ \n'
                                   'ופיתוח ת וכנית  להמשכיות עסקית.  ההוראה '
                                   'נסמכת על רגולציה קיימת ב מדינות רבות \n'
                                   'בעולם  מזה מספר שנים כגון: ארה"ב, קנדה, '
                                   'הונג קונג ואוסטרליה וכן על מסמך שפורסם \n'
                                   'על ידי פורום משותף בראשות ועדת באזל. \n'
                                   '6.  עקרון מרכזי שנקבע בהוראה הינו ש הוצאת '
                                   'פעי

In [31]:
ara = res['matches'][0]['metadata']['text']
ara


'אשראי, או לצורך  ביצוע התאמות לשיעורי הפסד היסטוריים. \n31. תאגיד בנקאי יכול להשתמש בשיטות שונות לקיבוץ חובות , לצורך הערכה ומדידה של סיכוני \nאשראי, בכפוף לנדרש בהוראות הדיווח לציבור . לדוגמה, חובות יכולים להיות מקובצים על  \nבסיס אחד, או יותר, מהמאפיינים הבאים: אומדן ההסתברות לכשל , דירוגי אשראי, סוג מוצר \nהאשראי, פלח שוק, מיקום גיאוגרפי, סוג הביטחונו ת ונתוני  פיגור בפירעון האשראי. מודלים  \nמתוחכמים יותר להערכת סיכוני אשראי ושיטות  להערכת תזרימי מזומנים עתידיים צפויים, \nוכן תהליכי דירוג אשראי, יכולים לשלב כמה ממאפיינים אלה. \n32.  מדיניות ונהלים נאותים להערכה ולמדידה של סיכוני אשראי אינם תלויים במטרה לשמה \nמבוצעות מדידת ההפרשות או הערכת הפסדי האשראי. כלומר, אותה מערכת נאותה להערכת \nסיכוני אשראי מספקת את המידע או התוצרים, אשר ישמשו למדידת הפסדי האשראי לצורך \nהערכת סיכוני אשראי, לטיפול ה חשבונאי, וכן למטרת הערכת הלימות ההון של התאגיד \nהבנקאי לפי גישת הדירוגים הפנימיים (כאשר מיושמת). זה יכול להביא לכך שהערכות דומות'

In [167]:
for match in res['matches']:
    print(f"score: {match['score']:.2f} test: {match['metadata']['text']} \n\n\n\n")

score: 0.87 test: על יציבות חברת הבת. בנוסף, המדיניות צריכה לטפל ב גורמי הבקרה החשובים ביחס לסקירת 
האשראי הפרטני ת, וביחס ל פיזור נאות של התיק בכללותו. 
 
02. ""סיכון מדינה" ו"סיכון העברה: 
א. כאשר תאגיד בנקאי עוסק במתן אשראי בינלאומי, הוא נוטל על עצמו, בנוסף  על סיכון 
האשראי הרגיל, גם סיכון הקשור לתנאים המתקיימים במדינתו של הלווה, או של הצד 
הנגדי לעסקה. "סיכון מדינה " כולל מגוון סיכונים ה קשורים לסביבה הכלכלית,  
הפוליטית והחברתית של אותה מדינה, אשר יש אפשרות ש הם ישפיעו על חובות  של המפקח על הבנקים: ניהול בנקאי תקין   [10( ] 11/23) 
ניהול סיכון אשראי  עמ' 11 - 311 
 
 גורמים מקומיים במדינה זו לתאגיד הבנקאי, ועל השקעות שנעשו בה. "סיכון העברה " 
מתמקד ביכולתו של הלווה להשיג מטבע חוץ בכמות הנדרשת לפירעון  חובותיו למלווים 
זרים, ולעמידה ב התחייבויות חוזיות נוספות שנלקח על עצמו . ב ביצוע כל עסקאותיו 
הבינלאומיות, תאגיד בנקאי  חייב להבין את השווקים הפיננסים הבינלאומיים , ואת 
האפשרות  של גלישת השפעות ממדינה אחת לאחרת או של התפשטות של השפעות  
שליליות ממדינה אחת  לכל האזור לו היא שייכת. 

In [12]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [2]:
load_qa_chain?

Signature:
load_qa_chain(
    llm: langchain_core.language_models.base.BaseLanguageModel,
    chain_type: str = 'stuff',
    verbose: Optional[bool] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    **kwargs: Any,
) -> langchain.chains.combine_documents.base.BaseCombineDocumentsChain
Docstring:
Load question answering chain.

Args:
    llm: Language Model to use in the chain.
    chain_type: Type of document combining chain to use. Should be one of "stuff",
        "map_reduce", "map_rerank", and "refine".
    verbose: Whether chains should be run in verbose mode or not. Note that this
        applies to all chains that make up the final chain.
    callback_manager: Callback manager to use for the chain.

Returns:
    A chain to use for question answering.
File:      c:\users\nirx_pc\documents\python projects\langchain_nir\lang_env\lib\site-packages\langchain\chains\question_answering\__init__.py
Type:      function

In [42]:
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
chain = load_qa_chain(llm, chain_type="stuff")

In [49]:
chain.invoke?

Signature:
chain.invoke(
    input: Dict[str, Any],
    config: Optional[langchain_core.runnables.config.RunnableConfig] = None,
    **kwargs: Any,
) -> Dict[str, Any]
Docstring:
Transform a single input into an output. Override to implement.

Args:
    input: The input to the runnable.
    config: A config to use when invoking the runnable.
       The config supports standard keys like 'tags', 'metadata' for tracing
       purposes, 'max_concurrency' for controlling how much work to do
       in parallel, and other keys. Please refer to the RunnableConfig
       for more details.

Returns:
    The output of the runnable.
File:      c:\users\nirx_pc\documents\python projects\langchain_nir\lang_env\lib\site-packages\langchain\chains\base.py
Type:      method

In [77]:
docsearch = Pinecone.from_text([t.page_content for t in text],embeddings_model,index_name=index_name_1536)

AttributeError: type object 'Pinecone' has no attribute 'from_text'

### Create conversation chain

In [171]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [206]:
def get_conversation_chain(vectorstore):
    llm = ChatOpenAI()
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        # retriever=vectorstore.as_retriver(),
        # retriever=vectorstore,
        memory=memory
    )
    return conversation_chain

In [207]:
text1 = res['matches'][0]['metadata']['text']

get_conversation_chain(text1)


TypeError: ConversationalRetrievalChain.from_llm() missing 1 required positional argument: 'retriever'

https://python.langchain.com/docs/get_started/quickstart/

In [28]:
import langchain_openai
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os


In [12]:

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


In [15]:
llm = ChatOpenAI(api_key=openai_api_key, model="gpt-4-turbo")
# llm2 = ChatOpenAI(api_key=openai_api_key, model="gpt-4-turbo-2024-04-09")

In [54]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001FFA589F450>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FFA4317B10>, model_name='gpt-4-turbo', openai_api_key=SecretStr('**********'), openai_proxy='')

In [55]:
llm2

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001FFA809FC50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FFA811F790>, model_name='gpt-4-turbo-2024-04-09', openai_api_key=SecretStr('**********'), openai_proxy='')

In [24]:
query = "with python, how to use langchain_openai with text of my own and query strin"

In [16]:
llm.invoke(query)

AIMessage(content='As of my last update in January 2023, `langchain_openai` doesn\'t directly refer to a specific, well-known library or API in mainstream use for integrating OpenAI\'s models. However, it is possible that you are referring to a custom or lesser-known implementation or simply asking how to use OpenAI\'s API (from OpenAI, the creators of GPT models) with your own text and a query. I\'ll provide guidance on how to use OpenAI\'s API (such as GPT-3 or ChatGPT) with your own input text and a query string.\n\n### Step 1: Set Up an OpenAI API Account\n\nBefore you can start using the OpenAI API, you need to create an account and obtain API keys. Here’s how you can do it:\n\n1. Visit the OpenAI website and sign up for an account if you haven’t already.\n2. After logging in, navigate to the API section and generate an API key. Keep this key secure as it allows usage of the API which you will be billed for based on usage.\n\n### Step 2: Install Required Libraries\n\nYou\'ll need 

In [30]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a AI code instractor the need to present me the code with python, I am an experienced python programmer"),
    ("user", "{input}")
])

In [22]:
chain = prompt | llm 

In [25]:
chain.invoke({"input": query})

AIMessage(content='To use the `langchain_openai` library in Python to process your own text and a query string, you first need to install the library and then use it to create a model that can respond to queries based on the text you provide. Below, I provide a step-by-step guide on how to set this up.\n\n### Step 1: Install Langchain and OpenAI Libraries\n\nYou\'ll need Python installed on your system. If you haven\'t installed Python yet, download and install it from [python.org](https://www.python.org/). Then, you can install the `langchain` and `openai` libraries using pip:\n\n```bash\npip install langchain openai\n```\n\n### Step 2: Setup OpenAI API Key\n\nTo use the OpenAI models, you\'ll need an API key from OpenAI. You can obtain this by creating an account on the OpenAI website and subscribing to the API service. Once you have your API key, you should set it as an environment variable or securely include it in your script. For this example, I will show how to set it directly i

In [29]:
output_parser = StrOutputParser()

In [31]:
chain = prompt | llm | output_parser

In [33]:
chain.invoke({"input": query})

'To use the `langchain_openai` package for integrating OpenAI models with your custom text and a specific query, you first need to install the package and have access to the OpenAI API. Below, I\'ll guide you through setting up a simple Python script that sends a custom text and a query to an OpenAI model using `langchain_openai`.\n\n### Step 1: Install the Necessary Package\n\nFirst, ensure that you have the `langchain_openai` package installed. You can install it using pip:\n\n```bash\npip install langchain-openai\n```\n\n### Step 2: Set Up Your API Key\n\nMake sure you have an API key from OpenAI. This key is necessary to authenticate your requests.\n\n### Step 3: Write the Python Code\n\nBelow is a simple Python script that demonstrates how to use `langchain_openai` to send a custom text along with a query string to the OpenAI model:\n\n```python\nfrom langchain.chains import SingleComponentChain\nfrom langchain.schema import Prompt\nfrom langchain.clients.openai import OpenAI\n\nd

In [14]:
dir(llm)

['Config',
 'InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__ror__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_updat

In [35]:
print('To use the `langchain_openai` package for integrating OpenAI models with your custom text and a specific query, you first need to install the package and have access to the OpenAI API. Below, I\'ll guide you through setting up a simple Python script that sends a custom text and a query to an OpenAI model using `langchain_openai`.\n\n### Step 1: Install the Necessary Package\n\nFirst, ensure that you have the `langchain_openai` package installed. You can install it using pip:\n\n```bash\npip install langchain-openai\n```\n\n### Step 2: Set Up Your API Key\n\nMake sure you have an API key from OpenAI. This key is necessary to authenticate your requests.\n\n### Step 3: Write the Python Code\n\nBelow is a simple Python script that demonstrates how to use `langchain_openai` to send a custom text along with a query string to the OpenAI model:\n\n```python\nfrom langchain.chains import SingleComponentChain\nfrom langchain.schema import Prompt\nfrom langchain.clients.openai import OpenAI\n\ndef main():\n    # Replace \'your_openai_api_key\' with your actual OpenAI API key\n    openai_api_key = \'your_openai_api_key\'  \n    \n    # Initialize the OpenAI client\n    openai_client = OpenAI(api_key=openai_api_key)\n\n    # Create a single component chain using the OpenAI client\n    chain = SingleComponentChain(component=openai_client)\n\n    # Example text and query\n    custom_text = "The quick brown fox jumps over the lazy dog."\n    query = "What is the moral of the story?"\n\n    # Compose your prompt\n    prompt = Prompt(f"{custom_text}\\n\\n###\\n\\n{query}\\n")\n\n    # Run the chain with the given prompt\n    result = chain.run(prompt)\n\n    # Print the result\n    print("Response:", result)\n\nif __name__ == "__main__":\n    main()\n```\n\n### Explanation:\n\n- **OpenAI Client**: First, you initialize the `OpenAI` client with your API key. This client is responsible for interacting with the OpenAI API.\n  \n- **Single Component Chain**: The `SingleComponentChain` is used here for simplicity, which consists of a single component (in this case, the OpenAI client).\n\n- **Prompt Composition**: The prompt is composed by appending your query to the custom text, separated by some markup (e.g., `###`) to clearly define the separation in the context for the model.\n\n- **Running the Chain**: The chain is then executed with the composed prompt, and the result is obtained.\n\n- **Output**: Finally, the output of the model\'s response is printed.\n\n### Step 4: Run Your Script\n\nMake sure that everything is set up correctly, and then run your Python script. You should see the response from the OpenAI model based on your custom text and query.\n\nThis is a basic example to get you started. Depending on your specific needs, `langchain_openai` offers more complex configurations and possibilities, such as using multiple components in a chain for more sophisticated workflows.')

To use the `langchain_openai` package for integrating OpenAI models with your custom text and a specific query, you first need to install the package and have access to the OpenAI API. Below, I'll guide you through setting up a simple Python script that sends a custom text and a query to an OpenAI model using `langchain_openai`.

### Step 1: Install the Necessary Package

First, ensure that you have the `langchain_openai` package installed. You can install it using pip:

```bash
pip install langchain-openai
```

### Step 2: Set Up Your API Key

Make sure you have an API key from OpenAI. This key is necessary to authenticate your requests.

### Step 3: Write the Python Code

Below is a simple Python script that demonstrates how to use `langchain_openai` to send a custom text along with a query string to the OpenAI model:

```python
from langchain.chains import SingleComponentChain
from langchain.schema import Prompt
from langchain.clients.openai import OpenAI

def main():
    # Replace 

### Retrieval Chain

In [45]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document


In [37]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

In [40]:
docs = loader.load()
embeddings = OpenAIEmbeddings()

In [42]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [51]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [52]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001FFA589F450>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001FFA4317B10>, model_name='gpt-4-turbo', openai_api_key=SecretStr('**********'), openai_proxy='')

In [47]:
answer = document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})
print(answer)

Langsmith can help with testing by allowing you to visualize test results, making it easier to understand and analyze the outcomes of tests.


In [48]:
from langchain.chains import create_retrieval_chain

In [49]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [50]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith assists with testing LLM applications in several ways:

1. **Creation of Test Cases**: Developers can create datasets comprising inputs and reference outputs to form test cases. These cases can be uploaded in bulk, created on the fly, or exported from application traces.

2. **Running Custom Evaluations**: The platform allows for both LLM-based and heuristic-based evaluations to assess test results. This helps in evaluating how well the application is performing according to predefined benchmarks.

3. **Comparison View**: LangSmith provides a comparison view that allows developers to assess different application versions against initial test cases. This helps in identifying whether recent changes have caused regressions in test scores.

4. **Beta Testing and Feedback Collection**: During the beta testing phase, developers can collect extensive real-world data on application performance. Feedback and annotations gathered during this phase are crucial in refining the test cases